In [1]:
import os
import sys
import time
from datetime import date
from datetime import datetime
import time
import json
import platform

import stoneburner
#//*** Custom Functions:
#//*** mr_clean_text(input_series)
#//*** tokenize_series(input_series)
#//*** remove_stop_words(input_series)

# //*** Imports and Load Data
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

#//*** Use the whole window in the IPYNB editor
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
#subreddits = ["wallstreetbets", "stocks", "wallstreetbetsOGs", "spacs", "investing", "pennystocks", "stockmarket", "options", "robinhoodpennystocks", "wallstreetbetsnew", "smallstreetbets"]
filepath = "./data/"
#filename_suffix = "_comments.csv.zip"
#//*** Maximize columns and rows displayed by pandas
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

In [3]:
#//*** Path to file aggregate
input_filename = "./data/processed_reddit_v4_1min.csv.zip"

input_filename = "./data/processed_reddit_v4_1min_test.pkl"
input_filename = "./ignore_folder/processed_reddit_v4_amc_1min_csv.zip"

raw_df = pd.read_csv(input_filename)

del raw_df['created_utc']
raw_df

,body,time,open,high,low,close,volume
0,Index fund tracks the performance of an stock ...,2019-07-24 09:00:00,4.09,4.09,4.09,4.09,406
1,No he’s just being sarcastic,2019-07-24 09:00:00,4.09,4.09,4.09,4.09,406
2,I was in contact with Schwab and they weren't ...,2019-07-24 09:00:00,4.09,4.09,4.09,4.09,406
3,Reelection is never a concern for the communis...,2019-07-24 09:00:00,4.09,4.09,4.09,4.09,406
4,"For American brokerages, up to $500,000 are SI...",2019-07-24 09:00:00,4.09,4.09,4.09,4.09,406
...,...,...,...,...,...,...,...
4355077,That's how you value a house? Cars are depreci...,2021-06-30 19:59:00,214.50,214.50,214.50,214.50,134
4355078,$dare yolo,2021-06-30 19:59:00,214.50,214.50,214.50,214.50,134
4355079,Back to normal not there yet,2021-06-30 19:59:00,214.50,214.50,214.50,214.50,134
4355080,💎👐🚀🚀🚀🚀,2021-06-30 19:59:00,214.50,214.50,214.50,214.50,134


In [5]:
raw_df['body'] = raw_df['body'].astype('str')

In [6]:
#//************************************************
#//*** Combine Body Text for each Stock Time
#//************************************************
tdf = pd.DataFrame()

print(len(raw_df['time'].unique()))
print("Starting Body Aggregation....")
start_time = time.time()
for group in raw_df.groupby('time'):
    
    #//*** Build aa temporary Series by grabbing the first line of the group[1] dataframe
    #//*** Only Body will update, the rest are static
    loop_series = group[1].iloc[0].copy()

    
    #//*** Combine all body strings into a single string, add to loop_series['body']
    loop_series['body'] = " ".join(list(group[1]['body']))
    #//*** Build a dataframe to hold the Series.
    #//*** Add Dataframe to tdf
    #print(loop_series)
    tdf = pd.concat ([tdf,pd.DataFrame(data=[loop_series], columns = list(group[1].columns))])

print(f"Built: {round(time.time()-start_time,2)}")
tdf

190326
Starting Body Aggregation....
Built: 2157.89


,body,time,open,high,low,close,volume
0,Index fund tracks the performance of an stock ...,2019-07-24 09:00:00,4.09,4.090,4.0900,4.090,406
11,It’s a combination of options but I made big o...,2019-07-24 09:35:00,4.14,4.150,4.1300,4.150,10116
12,Idk about you but I know several people with H...,2019-07-24 09:36:00,4.14,4.170,4.1385,4.170,16449
13,Thanks.. Will check them out,2019-07-24 09:37:00,4.17,4.180,4.1540,4.175,15605
14,I'm waiting until Monday to sell a covered cal...,2019-07-24 09:40:00,4.19,4.195,4.1700,4.180,42314
...,...,...,...,...,...,...,...
4355042,It’s always possible but I like to give compan...,2021-06-30 19:19:00,214.78,214.780,214.7800,214.780,100
4355055,Did business in China so I am jaded When money...,2021-06-30 19:40:00,214.80,214.800,214.8000,214.800,176
4355064,"I automatically downvoted on the image alone, ...",2021-06-30 19:54:00,214.34,214.340,214.3200,214.320,811
4355065,"So how do I get my 12,000 I lost from these fo...",2021-06-30 19:55:00,214.15,214.150,214.0000,214.000,1499


In [7]:
#//************************************************
#//*** Clean text body
#//************************************************

tdf['token'] = stoneburner.remove_stop_words(stoneburner.tokenize_series(stoneburner.mr_clean_text(tdf['body'],{"remove_empty":False})))
print("Done")

#//*** Detokenize the clean column as tfidf
tdf['tfidf'] = tdf['token'].apply(lambda x: ' '.join(x))


remove_empty False
Text Cleaning Time: 51.60511350631714
Tokenize Time: 185.8945128917694
Stop Words Time: 80.67693901062012
Done


In [8]:
#//*** Build tfidf
from sklearn.feature_extraction.text import TfidfVectorizer

#loop_list.append(tfidf.fit_transform(input_df['tfidf']))
tfidf_matrix = []
tfidf_list = []
tfidf = TfidfVectorizer()

print("Starting tfidf....")
start_time = time.time()
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(tdf['tfidf'])


print(f"Built: {round(time.time()-start_time,2)}")

print(tfidf_matrix)

Starting tfidf....
Built: 30.07
  (0, 213879)	0.06825788227739096
  (0, 281715)	0.07204481598012172
  (0, 81059)	0.05259839960026004
  (0, 318080)	0.04426673606716234
  (0, 46180)	0.0544448974837595
  (0, 376399)	0.059410997838819424
  (0, 342504)	0.037815124726759325
  (0, 377129)	0.153452124936028
  (0, 304783)	0.04689966061241974
  (0, 269113)	0.030196730748772528
  (0, 348772)	0.03496765799589904
  (0, 254031)	0.04269663086447837
  (0, 285339)	0.05369104737503451
  (0, 414213)	0.030016615624407838
  (0, 246123)	0.022711937020156364
  (0, 175607)	0.04759581796901236
  (0, 159022)	0.10728744447856593
  (0, 353676)	0.05623604783842471
  (0, 154872)	0.06791375624971967
  (0, 86847)	0.10079880960760651
  (0, 85168)	0.025869866725832594
  (0, 163267)	0.030016615624407838
  (0, 371421)	0.02850827632121679
  (0, 234411)	0.05290385234649163
  (0, 251032)	0.039401664005252586
  :	:
  (190325, 376816)	0.13077897875799496
  (190325, 253776)	0.06608576337966365
  (190325, 223290)	0.034453540314

In [9]:
import pickle
output_filename = "./data/processed_reddit_v4_1min_tfidf.pkl"

with open(output_filename, 'wb') as f:
    pickle.dump(tfidf, f)

output_filename = "./data/processed_reddit_v4_1min_tfidf_matrix.pkl"
with open(output_filename, 'wb') as f:
    pickle.dump(tfidf_matrix, f)


In [10]:
output_filename = "./data/processed_reddit_v4_1min_amc_stock_comments.pkl"

#//*** Cleanup columns for storage
if 'body' in tdf.columns:
    del tdf['body']
    del tdf['tfidf']

tdf.to_pickle(output_filename)

In [17]:
output_filename = "./data/processed_reddit_v4_1min_testdata_tfidf.pkl"
infile = open(output_filename,'rb')
t_tdif = pickle.load(infile)
infile.close()
print(t_tdif.get_params())
print(t_tdif.get_feature_names())


{'analyzer': 'word', 'binary': False, 'decode_error': 'strict', 'dtype': <class 'numpy.float64'>, 'encoding': 'utf-8', 'input': 'content', 'lowercase': True, 'max_df': 1.0, 'max_features': None, 'min_df': 1, 'ngram_range': (1, 1), 'norm': 'l2', 'preprocessor': None, 'smooth_idf': True, 'stop_words': None, 'strip_accents': None, 'sublinear_tf': False, 'token_pattern': '(?u)\\b\\w\\w+\\b', 'tokenizer': None, 'use_idf': True, 'vocabulary': None}


In [10]:
infile = open(output_filename,'rb')
t_tdif_matrix = pickle.load(infile)
infile.close()
print(t_tdif_matrix)

  (0, 8133)	0.08745808745100456
  (0, 11133)	0.07327893552266958
  (0, 2875)	0.05745903186031536
  (0, 12858)	0.05009304185394831
  (0, 1271)	0.059828786385647564
  (0, 15883)	0.06475547596357362
  (0, 14146)	0.04850398207814858
  (0, 15909)	0.0967335584413286
  (0, 12160)	0.055511687856221854
  (0, 10623)	0.037181240474684976
  (0, 14506)	0.04293238130669861
  (0, 10000)	0.051679345634732955
  (0, 11281)	0.05792547524640468
  (0, 17464)	0.03648588477406036
  (0, 9638)	0.03109594680876921
  (0, 6328)	0.05332397563086158
  (0, 5387)	0.08913682801099229
  (0, 14747)	0.05538498395581354
  (0, 5053)	0.06446319823408891
  (0, 3136)	0.08154009758065602
  (0, 3067)	0.03470696153058779
  (0, 5639)	0.036828048837588646
  (0, 15687)	0.03635195522166141
  (0, 9140)	0.0552597281316865
  (0, 9863)	0.04670937342947679
  :	:
  (5008, 8008)	0.21328624075614527
  (5008, 12039)	0.17110212787774884
  (5008, 6375)	0.14488634710955123
  (5008, 15717)	0.12363931585752246
  (5008, 7543)	0.18153131157240643
 